In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization as BN, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K


In [2]:
pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.2-cp36-cp36m-win_amd64.whl (6.8 MB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.5.4-cp36-cp36m-win_amd64.whl (31.2 MB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import glob
import random

In [11]:
import scipy

In [12]:
from tensorflow.keras.layers import BatchNormalization as BN

In [13]:
#Array that will store data and its labels
data_train   = []
labels_train = []
data_test    = []
labels_test  = []

In [14]:
#Dimension of image limit
img_dims = (96, 96, 3)

In [15]:
#load image files from dataset
image_files_train = [f for f in glob.glob(r'D:\Face_detection\Training' + "/**/*", recursive=True) if not os.path.isdir(f)]
image_files_test = [f for f in glob.glob(r'D:\Face_detection\Validation' + "/**/*", recursive=True) if not os.path.isdir(f)]

In [16]:
random.shuffle(image_files_train)
random.shuffle(image_files_test)

In [17]:
#Converting images to array and labelling the categories
def convert(image_files):
    data = []
    for img in image_files:
        image = cv2.imread(img)
    
        image = cv2.resize(image, (img_dims[0], img_dims[1]))
        image = img_to_array(image)
        data.append(image)
    return data
#Saving label of image
def label(image_files):
    labels = []
    for img in image_files:
        label = img.split(os.path.sep)[-2] #D:\\Face_detection\\Training\\male\\135791.jpg.jpg
        if label == "female":
            label = 1
        else:
            label = 0
        
        labels.append([label]) #[[0], [1], [1], ....]
    return labels;

In [18]:
data_train = convert(image_files_train)
data_test  = convert(image_files_test)

In [19]:
labels_train = label(image_files_train)
labels_test  = label(image_files_test)

In [20]:
#Pre-Processing
data_train   = np.array(data_train, dtype="float")
labels_train = np.array(labels_train)
data_test    = np.array(data_test, dtype="float")
labels_test  = np.array(labels_test)

In [21]:
trainY = to_categorical(labels_train, num_classes = 2)
testY  = to_categorical(labels_test, num_classes = 2)

In [22]:
trainX = data_train
testX  = data_test

In [23]:
#Augmentation dataset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [24]:
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth) #(96, 96, 3) -> in our case
    chanDim = -1
    
    if K.image_data_format() == "channels_first": #Returns a string, either 'channel_first' or channels_last
        imputShape = (depth, height, width)
        chanDim = 1
        
    model.add(Conv2D(32, (3,3), padding = "Same", input_shape = inputShape))
    
    model.add(Activation("relu")) #as we are dealing with non-linear dataset
    
    model.add(BN(axis = chanDim))#It is possible that higer data value may overshadow the lower value hence normalisation is done.
    #BatchNormalisation is used to keep meanActivation close to '0' and standard deviation close to '1'.
    
    model.add(MaxPooling2D(pool_size = (2,2)))#maxpooling is used to reduce noice in dataset like: spots on face, beard, pimples, etc.
    #It looks only important features (eyes, noes, etc.) and remove others.
    
    model.add(Dropout(0.25)) # To avoid overfitting
    #25% neurons will be deactivated during front and backward propagation
    
    
    model.add(Conv2D(64, (3,3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BN(axis = chanDim))
    
    model.add(Conv2D(64, (3,3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BN(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BN(axis = chanDim))
    
    model.add(Conv2D(64, (3,3), padding = "same"))
    model.add(Activation("relu"))
    model.add(BN(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten()) #Flatten converts 2D into 1D
    model.add(Dense(1024)) #Add dense layer with 1024 neurons
    model.add(Activation("relu"))# relu is go to activation function when we do image classification
    model.add(BN())
    model.add(Dropout(0.5))
    
    model.add(Dense(classes))
    model.add(Activation("sigmoid")) #in outer we have given sigmod as it is probability based function ad give value from 0 to 1
    
    return model

In [25]:
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2], classes=2)

In [26]:
#compile the model
epochs = 70
lr = 1e-3
batch_size = 25

opt = Adam(learning_rate = lr, decay = lr/epochs) #Adam is optimizer->during back propagation how the weight should be updated
model.compile(loss="binary_crossentropy", optimizer=opt, metrics = ["accuracy"])

In [19]:
pip install scipy

In [28]:
import scipy

In [29]:
#train the model
'''H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
              steps_per_epoch = len(trainX)//batch_size,
              epochs = epochs, verbose =1)'''
H = model.fit(trainX, trainY, batch_size=batch_size, epochs = epochs)

Train on 19440 samples
Epoch 1/70
19440/19440 [==============================] - 41s 2ms/sample - loss: 0.3073 - accuracy: 0.8800
Epoch 2/70
19440/19440 [==============================] - 31s 2ms/sample - loss: 0.1951 - accuracy: 0.9243
Epoch 3/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1689 - accuracy: 0.9352
Epoch 4/70
19440/19440 [==============================] - 31s 2ms/sample - loss: 0.1642 - accuracy: 0.9402
Epoch 5/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1659 - accuracy: 0.9376
Epoch 6/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1382 - accuracy: 0.9479
Epoch 7/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1283 - accuracy: 0.9527
Epoch 8/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1173 - accuracy: 0.9569
Epoch 9/70
19440/19440 [==============================] - 30s 2ms/sample - loss: 0.1126 - accuracy: 0.9592
Epoch 10/70
19

In [31]:
model.save("face_recognition.model")

INFO:tensorflow:Assets written to: face_recognition.model\assets
